In [3]:
from utils.analysis.tools import simDir, periodicity_plot
from pathlib import Path
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm import tqdm
from utils import plot

In [4]:
class periodicityPhaseDiagram:
    def __init__(self,
                 directory: str,
                 sweep_file: str = "sweep_parameters.txt"):
        self.sweep_directory = Path(directory)
        self.sweep_file = self.sweep_directory / sweep_file
        self.sweep_parameters = [line.split(",")[0] for line in self.sweep_file.read_text().splitlines()]

    def extract_data(self):
        simdir_paths = [file.parent for file in self.sweep_directory.glob("./*/input_params.txt")]
        processes = mp.Pool(8-1)
        self.results = tqdm(processes.imap(self.worker,simdir_paths),total=len(simdir_paths))
        self.df = pd.DataFrame(self.results)
    
    def worker(self,worker_input):
        simdir_path = worker_input
        sim = simDir(simdir_path)
        param_values = sim.params
        relevant_params = {parameter: str(param_values[parameter]) for parameter in self.sweep_parameters}
        sim.run()
        sim.rna()
        sim.condensate()
        sim.periodicity(15000)
        period = np.diff(sim.time[sim.peaks],axis=0).mean()
        shortest = np.min([sim.peaks.shape,sim.troughs.shape])
        peaks = sim.peaks[:shortest]
        troughs = sim.troughs[:shortest]
        com_height = np.abs(sim.com[peaks,0]-sim.com[troughs,0]).mean()
        radius_variance_height = np.abs(sim.radius_variance[peaks]-sim.radius_variance[troughs]).mean()
        eccentricity_height = np.abs(sim.eccentricity[peaks]-sim.eccentricity[troughs]).mean()
        rna_height = np.abs(sim.rna_amount[peaks]-sim.rna_amount[troughs]).mean()
        if troughs.size == 0 or peaks.size == 0:
            return relevant_params | {"period": period,
                                  "com_height": com_height,
                                  "radius_variance_height": radius_variance_height,
                                  "eccentricity_height": eccentricity_height,
                                  "rna_height": rna_height,
                                  "xy_trough_end": np.nan,
                                  "xy_peak_end": np.nan,
                                  "c_trough_end": np.nan,
                                  "c_peak_end": np.nan}
        else:
            xy_trough_end = sim.xy[sim.mask[troughs[-1],:]]
            c_trough_end = sim.concentration_profile[0][troughs[-1],:][sim.mask[troughs[-1],:]]
            xy_peak_end = sim.xy[sim.mask[peaks[-1],:]]
            c_peak_end = sim.concentration_profile[0][peaks[-1],:][sim.mask[peaks[-1],:]]
            return relevant_params | {"period": period,
                                    "com_height": com_height,
                                    "radius_variance_height": radius_variance_height,
                                    "eccentricity_height": eccentricity_height,
                                    "rna_height": rna_height,
                                    "xy_trough_end": xy_trough_end,
                                    "xy_peak_end": xy_peak_end,
                                    "c_trough_end": c_trough_end,
                                    "c_peak_end": c_peak_end}

In [5]:
ppd = periodicityPhaseDiagram("/nfs/arupclab001/davidgoh/CoupledEPCondensates/workspace/05_TimeDelay/20240223_PhaseDiagramHillFaster")

In [6]:
ppd.extract_data()

 79%|███████▉  | 101/128 [09:55<02:04,  4.62s/it]

In [ ]:
ppd.df.to_pickle("20240223_PhaseDiagramHillFaster.pkl")

In [ ]:
df = pd.read_pickle("20240223_PhaseDiagramHillFaster.pkl")

In [ ]:
df["tau"] = df["tau"].astype(np.float64)
df["chiPR_tilde"] = df["chiPR_tilde"].astype(np.float64)
df["rna_height"] = df["rna_height"].astype(np.float64)

In [ ]:
variable = "rna_height"
rna_pivot = df.pivot_table(index="chiPR_tilde",columns="tau",values=variable)
xlabel = "Time delay"
ylabel = "Reentrant interactions"
fig, ax = plt.subplots()
sns.heatmap(rna_pivot,cmap="mako",ax=ax,cbar_kws={"label":r"$\Delta$RNA"})
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.legend()
for spine in ax.spines:
    ax.spines[spine].set_visible(True)

In [ ]:
variable = "period"
pivot = df.pivot_table(index="chiPR_tilde",columns="tau",values=variable)
pivot[rna_pivot<2] = np.nan
xlabel = "Time delay"
ylabel = "Reentrant interactions"
fig, ax = plt.subplots()
sns.heatmap(pivot,cmap="mako",ax=ax,cbar_kws={"label":"Mean period"})
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.legend()
for spine in ax.spines:
    ax.spines[spine].set_visible(True)

In [ ]:
df.columns

In [ ]:
variable = "radius_variance_height"
pivot = df.pivot_table(index="chiPR_tilde",columns="tau",values=variable)
xlabel = "Time delay"
ylabel = "Reentrant interactions"
fig, ax = plt.subplots()
sns.heatmap(pivot,cmap="mako",ax=ax,cbar_kws={"label":r"$\Delta \mathrm{Var}(r)$"})
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.legend()
for spine in ax.spines:
    ax.spines[spine].set_visible(True)

In [ ]:
variable = "eccentricity_height"
pivot = df.pivot_table(index="chiPR_tilde",columns="tau",values=variable)
xlabel = "Time delay"
ylabel = "Reentrant interactions"
fig, ax = plt.subplots()
sns.heatmap(pivot,cmap="mako",ax=ax,cbar_kws={"label":r"$\Delta$Eccentricity"})
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.legend()
for spine in ax.spines:
    ax.spines[spine].set_visible(True)

In [ ]:
variable = "com_height"
pivot = df.pivot_table(index="chiPR_tilde",columns="tau",values=variable)
xlabel = "Time delay"
ylabel = "Reentrant interactions"
fig, ax = plt.subplots()
sns.heatmap(pivot,cmap="mako",ax=ax,cbar_kws={"label":r"$\Delta$COM"})
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.legend()
for spine in ax.spines:
    ax.spines[spine].set_visible(True)

In [ ]:
df.columns

In [ ]:
df.loc[df["rna_height"]<2,"period"] = np.nan

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(df,x="tau",y="period",hue="chiPR_tilde",palette="crest",
             marker="o",legend="full",ax=ax)
taus = np.arange(25,450,50)
ax.plot(taus,2*taus,color="grey",linestyle="--",)
ax.annotate(text=r"$y=2\tau$",xy=(100,240),rotation=np.rad2deg(np.arctan(1/2)))
ax.legend(title="Reentrant interactions")
ax.set_ylabel("Mean period")
ax.set_xlabel("Time Delay")
ax.legend(bbox_to_anchor=(1, 1))


In [ ]:
df["period_diff"] = df["period"]-2*df["tau"]
df.loc[df["rna_height"]<2,"period_diff"] = np.nan

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(df,x="tau",y="period_diff",hue="chiPR_tilde",palette="crest",
             marker="o",legend="full",ax=ax)
taus = np.arange(25,450,50)
ax.legend(title="Reentrant interactions")
ax.set_ylabel(r"Mean period - 2 $\times$ time delay")
ax.set_xlabel("Time Delay")
ax.legend(bbox_to_anchor=(1, 1),title="Reentrant\ninteractions")


In [ ]:
fig, ax = plt.subplots()
sns.lineplot(df,x="chiPR_tilde",y="period_diff",hue="tau",palette="crest",
             marker="o",legend="full",ax=ax)
taus = np.arange(25,450,50)
ax.legend(title="Reentrant interactions")
ax.set_ylabel(r"Mean period - 2 $\times$ time delay")
ax.set_xlabel("Reentrant Interactions")
ax.legend(bbox_to_anchor=(1, 1),title="Time\ndelay")


In [ ]:
sim = simDir("/nfs/arupclab001/davidgoh/CoupledEPCondensates/workspace/05_TimeDelay/20240223_PhaseDiagramHillFaster/M1_1.0_b_-0.25_g_-0.1_c_0.03_k_0.05_kp_0.2_c1_3.53_sw_1.0_sr_2.5_cn_5.5_l_10_M3_0.1_kt_0.0_rl_0.0_wd_0.0_t_350.0_kd_1.0")
sim.run()
sim.condensate()
sim.rna()

In [ ]:
pivot

In [ ]:
def periodicity_plot(sim,threshold,leftlim=0,rightlim=20000):
    sim.periodicity(threshold)
    rna = np.ravel(sim.rna_amount)
    com = np.ravel(sim.com[:,0])
    time = np.raval(sim.time)
    if time.size > com.size:
        time = time[:com.size]
    fig,axes = plt.subplots(4,1,sharex=True)
    fig.set_size_inches(5,3)
    axes[0].plot(time,rna)
    start = np.argmin((time-threshold)**2)
    axes[0].scatter(time[sim.peaks],rna[sim.peaks],alpha=0.3)
    axes[0].scatter(time[sim.troughs],rna[sim.troughs],alpha=0.3)
    diffs = np.diff(time[sim.peaks],axis=0)
    locst = (time[sim.peaks][:-1] + time[sim.peaks][1:])/2
    for i in range(len(diffs)):
        axes[0].annotate(f"{diffs[i].item():.0f}",(locst[i],0),ha='center',rotation=90,
                         va='bottom')
    axes[1].plot(time,com)
    axes[2].plot(time,np.var(sim.radius,axis=1))
    axes[3].plot(time,sim.eccentricity)
    [ax.axvline(i, color='grey', ls='dashed') for i in time[sim.peaks] for ax in axes]
    axes[3].set_xlabel("Time")
    axes[0].set_ylabel("RNA\nAmount")
    axes[1].set_ylabel("Center of\nMass")
    axes[2].set_ylabel("Variance of\nRadius")
    axes[3].set_ylabel("Eccentricity")
    axes[3].set_xlim(left=leftlim,right=rightlim)
    return fig,axes,sim.peaks,sim.troughs

In [ ]:
periodicity_plot(sim,threshold=10000)

In [ ]:
pivot

In [ ]:
import matplotlib as mpl
c_white_trans = mpl.colors.colorConverter.to_rgba('white',alpha = 0)
c_white= mpl.colors.colorConverter.to_rgba('white',alpha = 1)
cmap_white = mpl.colors.LinearSegmentedColormap.from_list('rb_cmap',[c_white_trans,c_white],512)
c_black_trans = mpl.colors.colorConverter.to_rgba('black',alpha = 0)
c_black= mpl.colors.colorConverter.to_rgba('black',alpha = 1)
cmap_black = mpl.colors.LinearSegmentedColormap.from_list('rb_cmap',[c_black_trans,c_black],512)
def shape(xy,c,ax,cmap=cmap_rb):
    x = xy[:,0]
    y = xy[:,1]
    ax.tricontourf(x,y,c,cmap=cmap)
    ax.set_aspect(1)
    for spine in ax.spines:
        ax.spines[spine].set_visible(False)
        ax.tick_params(
        axis='both',
        which='both',
        bottom=False,
        left=False,
        labelbottom=False,
        labelleft=False)

In [ ]:
mask_pivot.loc[::-1]

In [ ]:
mask_pivot = df.pivot(index="chiPR_tilde",columns="tau",values=["xy_peak_end", "c_peak_end"])
fig,axes=plt.subplots(len(mask_pivot["xy_peak_end"].index),len(mask_pivot["xy_peak_end"].columns))
fig.set_size_inches(18,15)
for ax,arr_xy,arr_c,rna_height in zip(axes.flatten(),
                           mask_pivot["xy_peak_end",].to_numpy().flatten(),
                           mask_pivot["c_peak_end",].to_numpy().flatten(),
                           rna_pivot.to_numpy().flatten()):
    if rna_height > 2:
        shape(arr_xy,arr_c,ax,cmap_white)
    else:
         shape(arr_xy,arr_c,ax,cmap_black)
plt.savefig("mask_alpha_peak.png", transparent=True, dpi=300, bbox_inches='tight')

In [ ]:
mask_pivot = df.pivot(index="chiPR_tilde",columns="tau",values=["xy_trough_end", "c_trough_end"])
fig,axes=plt.subplots(len(mask_pivot["xy_trough_end"].index),len(mask_pivot["xy_trough_end"].columns))
fig.set_size_inches(18,15)
for ax,arr_xy,arr_c,rna_height in zip(axes.flatten(),
                           mask_pivot["xy_trough_end",].to_numpy().flatten(),
                           mask_pivot["c_trough_end",].to_numpy().flatten(),
                           rna_pivot.to_numpy().flatten()):
    if rna_height > 2:
        shape(arr_xy,arr_c,ax,cmap_white)
    else:
         shape(arr_xy,arr_c,ax,cmap_black)
plt.savefig("mask_alpha_trough.png", transparent=True, dpi=300, bbox_inches='tight')